Try to use Colab

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive
%rm -f -r mlmech
! git clone https://github.com/jfra1397/mlmech.git
%cd mlmech

In [ ]:
!git checkout main
!git pull

Training starts here

In [1]:
import custom
import create_model
from importlib import reload

In [2]:
reload(custom)
from custom import *

from load_data_lena import CustomDataGenerator


img_dir = "images/"
mask_dir = "labels/"
image_extension = ".png"
mask_extension = ".png"
img_size = (256, 256)
#batch_size = 16
horizontal_split = 12
vertical_split = 1

train, validation = CustomDataGenerator.generate_data(batch_size, img_dir, mask_dir,
                                                        horizontal_split, vertical_split, image_extension, mask_extension, 
                                                        preprocess_fcn, validation_split=0.1, flip=True, shift = shift, onelabel=onelabel, seed=seed)

Number of samples:  5400
Classes: [0 1]
Number of samples:  600
Classes: [0 1]


In [3]:
#train.plot_batch(3)

img, mask = train.__getitem__(2)
print(img.min())
print(img.max())
print(mask.min())
print(mask.max())
print(train.classes)

-1.0
1.0
0.0
1.0
[0 1]


In [4]:
#from keras import backend
#K = keras.backend.backend()
import tensorflow as tf

reload(custom)
from custom import *

reload(create_model)
#from create_model import generate_model

import tensorflow.keras.losses as losses
from tensorflow.keras import backend

#model = generate_model()
#from segnet_model2 import create_segnet
#from segnet_model import create_segnet
#from segnet_model_lenaWithLayers_ import create_segnet
from segnetSimple import create_segnet
#from segnet_vgg16_001 import SegNet_VGG16

if onelabel == True:
    n_labels = 1
else:
    n_labels = 3
model = create_segnet(input_shape=(256,256,3), n_labels=n_labels, output_mode="sigmoid")
#model = SegNet_VGG16(input_shape=(256,256,3), num_classes=n_labels)

#model = create_segnet(input_shape=(256,256,3), n_labels=n_labels, output_mode="softmax")
#model = create_segnet(input_shape=(256,256,3), n_labels=n_labels, output_mode="softmax")
#model = SegNet(input_shape=(256,256,3), classes=n_labels)
#from segnet4 import segnet

#from unetsegnet import generate_model
#model = generate_model()


#model = segnet(input_shape=(256,256,3), n_labels=n_labels)

#opti = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer='adam',
                loss=loss,
                metrics=['accuracy'])

model.summary()
#print(model.optimizer.learning_rate.numpy())
#backend.set_value(model.optimizer.learning_rate, 0.01)
print(model.optimizer.learning_rate.numpy())

#from segnet_test import *

#test_max_pooling_argmax()

#test_max_unpooling()



Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 256, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [5]:
reload(custom)
from custom import *

history = model.fit(x=train, validation_data = validation, epochs=epochs, steps_per_epoch=steps_per_epoch, verbose=1, shuffle=True)#callbacks=[callback], verbose=1, shuffle=True)

Epoch 1/50
12/20 [=================>............] - ETA: 35s - loss: 5.7901 - accuracy: 0.6203

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,7))
plt.plot(history.history["loss"], lw=4, label="loss")
plt.plot(history.history["val_loss"], lw=4, label="val_loss")
plt.legend()
#plt.ylim(top=1.5, bottom=0.4)
plt.savefig("loss.png")

In [ ]:
images, masks = train.__getitem__(6)
preds = model.predict(x=images, verbose=1)
#print(preds[1])

train.plot_prediction(6, preds)

In [ ]:
reload(custom)
from custom import *

import pandas as pd
import os

os.mkdir(dir_name)

from shutil import copyfile
copyfile("custom.py", dir_name + "/custom.py")
copyfile("create_model.py", dir_name + "/create_model.py")


# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history.history) 

# save to json:  
hist_json_file = dir_name + '/history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

#save model
model.save(dir_name + "/model.tf", include_optimizer = True)
model.save_weights(dir_name + "/weights.tf")